In [1]:
import ee
print("Earth Engine API imported successfully!")

Earth Engine API imported successfully!


In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-damyadharmawardhana')


Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [4]:
import pandas as pd
ee.Initialize(project='ee-damyadharmawardhana')

# Define the area of interest (Peradeniya, Kandy).
aoi = ee.Geometry.Point([80.593016, 7.271481])

# Define the time range for the analysis.
start_date = '2010-01-01'
end_date = '2021-12-31'

# Load the ERA5 Daily dataset.
era5 = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .filterDate(start_date, end_date) \
    .select('mean_2m_air_temperature')

# Function to reduce the image collection to a time series.
def extract_temperature(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    temperature = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=10000
    ).get('mean_2m_air_temperature')
    return ee.Feature(None, {'date': date, 'temperature': temperature})

# Apply the function to the image collection.
temperature_series = era5.map(extract_temperature).filter(ee.Filter.notNull(['temperature']))

# Convert to a list of dictionaries.
temperature_list = temperature_series.getInfo()['features']

# Convert the list to a pandas DataFrame.
temperature_data = []
for item in temperature_list:
    date = item['properties']['date']
    temperature = item['properties']['temperature'] - 273.15  # Convert from Kelvin to Celsius
    temperature_data.append({'date': date, 'temperature': temperature})

df = pd.DataFrame(temperature_data)

# Save to CSV
df.to_csv('temperature_peradeniya_kandy_2006_2021.csv', index=False)

print('Temperature data exported to temperature_peradeniya_kandy_2006_2021.csv')

Temperature data exported to temperature_peradeniya_kandy_2006_2021.csv
